# [Day 6: Universal Orbit Map](https://adventofcode.com/2019/day/6)

## Part 1

You've landed at the Universal Orbit Map facility on Mercury. Because navigation in space often involves transferring between orbits, the orbit maps here are useful for finding efficient routes between, for example, you and Santa. You download a map of the local orbits (your puzzle input).

Except for the universal Center of Mass (`COM`), every object in space is in orbit around exactly one other object. An orbit looks roughly like this:

```
                  \
                   \
                    |
                    |
AAA--> o            o <--BBB
                    |
                    |
                   /
                  /
```

In this diagram, the object `BBB` is in orbit around `AAA`. The path that BBB takes around AAA (drawn with lines) is only partly shown. In the map data, this orbital relationship is written `AAA)BBB`, which means "`BBB` is in orbit around `AAA`".

Before you use your map data to plot a course, you need to make sure it wasn't corrupted during the download. To verify maps, the Universal Orbit Map facility uses **orbit count checksums** - the total number of **direct orbits** (like the one shown above) and **indirect orbits**.

Whenever `A` orbits `B` and `B` orbits `C`, then `A` **indirectly orbits** `C`. This chain can be any number of objects long: if `A` orbits `B`, `B` orbits `C`, and `C` orbits `D`, then `A` indirectly orbits `D`.

For example, suppose you have the following map:

```
COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
```

Visually, the above map of orbits looks like this:

```
        G - H       J - K - L
       /           /
COM - B - C - D - E - F
               \
                I
```                

In this visual representation, when two objects are connected by a line, the one on the right directly orbits the one on the left.

Here, we can count the total number of orbits as follows:
- `D` directly orbits `C` and indirectly orbits `B` and `COM`, a total of `3` orbits.
- `L` directly orbits `K` and indirectly orbits `J`, `E`, `D`, `C`, `B`, and `COM`, a total of `7` orbits.
- `COM` orbits nothing.

The total number of direct and indirect orbits in this example is **`42`**.

**What is the total number of direct and indirect orbits** in your map data?

In [6]:
from pathlib import Path

input_path = Path("..") / "inputs" / "day_06.txt"
inputs = input_path.read_text()
inputs = inputs.split("\n")

In [7]:
def count_orbits(orbits):
    orbits = dict(reversed(orbit.split(")")) for orbit in orbits)
    planets = {*orbits, *orbits.values()}
    
    distances = {}
    
    for planet in planets:
        planet_trace = planet
        counter = 0
        while planet_trace != "COM":
            planet_trace = orbits[planet_trace]
            counter += 1
        distances[planet] = counter
        
    return sum(distances.values())

### Test Cases

In [8]:
test_cases = [
    ([
        "COM)B", "B)C", "C)D", "D)E", "E)F", "B)G", "G)H", "D)I", 
        "E)J", "J)K", "K)L"
    ],
        42
    )
]

for input_, output in test_cases:
    assert count_orbits(input_) == output 

In [9]:
%%time
result = count_orbits(inputs)

CPU times: user 58.5 ms, sys: 0 ns, total: 58.5 ms
Wall time: 57.9 ms


In [10]:
print(f"Answer to Part 1: {result} ({result:,})")

Answer to Part 1: 308790 (308,790)


## Part 2

Now, you just need to figure out how many **orbital transfers** you (`YOU`) need to take to get to Santa (`SAN`).

You start at the object `YOU` are orbiting; your destination is the object `SAN` is orbiting. An orbital transfer lets you move from any object to an object orbiting or orbited by that object.

For example, suppose you have the following map:
```
COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN
```

Visually, the above map of orbits looks like this:

```
                          YOU
                         /
        G - H       J - K - L
       /           /
COM - B - C - D - E - F
               \
                I - SAN
```

In this example, `YOU` are in orbit around `K`, and `SAN` is in orbit around `I`. To move from `K` to `I`, a minimum of `4` orbital transfers are required:
- `K` to `J`
- `J` to `E`
- `E` to `D`
- `D` to `I`

Afterward, the map of orbits looks like this:

```
        G - H       J - K - L
       /           /
COM - B - C - D - E - F
               \
                I - SAN
                 \
                  YOU
```

**What is the minimum number of orbital transfers required** to move from the object `YOU` are orbiting to the object `SAN` is orbiting? (Between the objects they are orbiting - **not** between `YOU` and `SAN`.)

In [11]:
from collections import defaultdict

def count_orbital_injections(object_1, object_2, orbits):
    orbits = dict(reversed(orbit.split(")")) for orbit in orbits)
    visited = defaultdict(dict)

    for planet in [object_1, object_2]:
        distance = -1
        planet_trace = planet
        while planet_trace != "COM":
            planet_trace = orbits[planet_trace]
            distance += 1
            visited[planet][planet_trace] = distance
    common_planets = set(
        visited[object_1]
    ).intersection(visited[object_2])
    distances = [
        visited[object_1][planet] + visited[object_2][planet] 
        for planet in common_planets
    ]
    return min(distances)

### Test Cases

In [12]:
test_cases = [
    ([
        "COM)B", "B)C", "C)D", "D)E", "E)F", "B)G", "G)H", "D)I", 
        "E)J", "J)K", "K)L", "K)YOU", "I)SAN"
    ],
        "YOU", "SAN", 4
    )
]

for input_, obj_1, obj_2, output in test_cases:
    assert count_orbital_injections(obj_1, obj_2, input_) == output 

In [13]:
%%time
result = count_orbital_injections("YOU", "SAN", inputs)

CPU times: user 2.07 ms, sys: 0 ns, total: 2.07 ms
Wall time: 2.11 ms


In [15]:
print(f"Answer to Part 2: {result}")

Answer to Part 1: 472
